In [ ]:
#Gerekli Kütüphanelerin Yüklenmesi Yapılıyor
import numpy as np # linear algebra
import pandas as pd # Veri işleme
#Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns
#Makine Öğrenmesi Gereksimleri
# Ön işleme için
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Model seçimi için
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
# Makine öğrenmesi modelleri
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
#Sistem Kütüphaneleri
import os
import warnings
print(os.listdir("../input/biomechanical-features-of-orthopedic-patients"))

In [ ]:
#Üç sınıf içeren veri seti
dataset_3c=pd.read_csv("../input/biomechanical-features-of-orthopedic-patients/column_3C_weka.csv")
#Veri setinin ilk beş örneği(satırı) listelenecek. n varsayılan olarak 5'tir.  
dataset_3c.head()

In [ ]:
#İki sınıf içeren veri seti
dataset_2c=pd.read_csv("../input/biomechanical-features-of-orthopedic-patients/column_2C_weka.csv")
#Veri setinin ilk beş örneği(satırı) listelenecek.
dataset_2c.head()

In [ ]:
dataset_3c.tail()

In [ ]:
dataset_2c.tail()

In [ ]:
dataset_2c.describe()

In [ ]:
dataset_3c.describe()

In [ ]:
dataset_3c.info()

In [ ]:
def discrete_univariate(dataset, discrete_feature):
    fig, axarr=plt.subplots(nrows=1,ncols=2, figsize=(8,5))
      
    dataset[discrete_feature].value_counts().plot(kind="bar",ax=axarr[0])
    dataset[discrete_feature].value_counts().plot.pie(autopct="%1.1f%%",ax=axarr[1])
        
    plt.tight_layout()
    plt.show()

discrete_univariate(dataset=dataset_3c, discrete_feature="class")

In [ ]:
def continuous_univariate(dataset, continuos_feature):
    fig, ax=plt.subplots(nrows=2,ncols=3, figsize=(12,8))

    ax=ax.flatten()

    "pandas çizim fonksiyonları kullanılıyor"
    dataset_3c[continuos_feature].plot.hist(density=True,ax=ax[0])
    dataset_3c[continuos_feature].plot.kde(ax=ax[1])
    dataset_3c[continuos_feature].plot.hist(density=True,ax=ax[2])
    dataset_3c[continuos_feature].plot.kde(ax=ax[2])

    "Seaborn çizim fonksiyonları kullanılıyor"
    sns.distplot(a=dataset_3c[continuos_feature], kde=False, ax=ax[3])
    sns.distplot(a=dataset_3c[continuos_feature], hist=False, ax=ax[4])
    sns.distplot(a=dataset_3c[continuos_feature], ax=ax[5]) 
    
continuous_univariate(dataset=dataset_3c, continuos_feature="pelvic_incidence")

In [ ]:
def plot_categorical(dataset, categorical_feature, rows, cols):
    fig, axarr=plt.subplots(nrows=4,ncols=2, figsize=(15,10))
    features=dataset.columns.values[:-1]
    
    counter=0
    #sns.countplot(x=categorical_feature, data=dataset, ax=axarr[0,0])
    dataset['class'].value_counts().plot.bar(ax=axarr[0,0])
    dataset['class'].value_counts().plot.pie(autopct="%1.1f%%",ax=axarr[0,1])
    for i in range(rows):
        for j in range(cols):
            feature=features[counter]
            if (i==0 and j==0) or (i==0 and j==1):
                continue
            else:
                sns.swarmplot(x=categorical_feature,y=feature,
                             
                            data=dataset, 
                            ax=axarr[i, j])
            counter=counter+1
            if counter>=len(features):
                break
    
    plt.tight_layout()
    plt.show()

plot_categorical(dataset=dataset_3c, categorical_feature="class", rows=4, cols=2)

In [ ]:
plot_categorical(dataset=dataset_2c, categorical_feature="class", rows=4, cols=2)

In [ ]:
#Sınıflar ile özelliklerin ikilişkisini gösteren pairplot
sns.pairplot(dataset_2c, hue="class", markers=["o", "s"])

In [ ]:
#Sınıflar ile özelliklerin ikilişkisini gösteren pairplot
sns.pairplot(dataset_3c, hue="class", markers=["o", "s",'D'])

Korelasyon matrisi değişkenlerin birbiriyle olan ilişkini yönünü ve gücünü belirler. piarplot grafiğinin sunduğu ilişkiyi daha açık ve sayısal değerlerle sunar. Korelasyon matrisi veri setini bir bütün olarak yorumlamak için en iyi grafik türüdür.

In [ ]:
corr=dataset_3c.corr()
fig, ax=plt.subplots(1,1,figsize=(12,8))
sns.heatmap(corr,annot=True, linewidth=.5, ax=ax)

Yardımcı Fonksiyon: draw_multivarient_plot(dataset, rows, cols, plot_type) 

Veri setindeki özelliklerin, sınıf ile ilişkilerini yorumlayabilmek için;

Keman Grafiği (Violin Plot)

Kutu Grafiği (Box Plot)

Nokta Grafiği (Point Plot)

Bar Grafiği (Bar Plot) grafiklerine göz atalım.

In [ ]:
#Sınıflar ile özelliklerin ikilişkisini gösteren farklı grafikler;
#boxplot, violinplot, pointplot, barplot
def draw_multivarient_plot(dataset, rows, cols, plot_type):
    
    assert plot_type in ['violin', "box", "point", "bar"],"We dont have such as plot type:{}".format(plot_type)
    column_names=dataset.columns.values
    number_of_column=len(column_names)
    fig, axarr=plt.subplots(rows,cols, figsize=(22,16))
    
    counter=0
    for i in range(rows):
        for j in range(cols):
            if 'violin' in plot_type:
                sns.violinplot(x='class', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'box'in plot_type :
                sns.boxplot(x='class', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'point' in plot_type:
                sns.pointplot(x='class',y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'bar' in plot_type:
                sns.barplot(x='class',y=column_names[counter],data=dataset, ax=axarr[i][j])
                
            counter+=1
            if counter==(number_of_column-1,):
                break

In [ ]:
draw_multivarient_plot(dataset=dataset_2c, rows=2, cols=3,plot_type="violin") #KemanGrafiği

In [ ]:
draw_multivarient_plot(dataset=dataset_2c, rows=2, cols=3,plot_type="point") #NoktaGrafiği

In [ ]:
draw_multivarient_plot(dataset=dataset_2c, rows=2, cols=3,plot_type="box") #KutuGrafiği

In [ ]:
draw_multivarient_plot(dataset=dataset_2c, rows=2, cols=3,plot_type="bar") #BarGrafiği

In [ ]:
draw_multivarient_plot(dataset=dataset_3c, rows=2, cols=3,plot_type="violin")

In [ ]:
draw_multivarient_plot(dataset=dataset_3c, rows=2, cols=3,plot_type="point")

In [ ]:
draw_multivarient_plot(dataset=dataset_3c, rows=2, cols=3,plot_type="box")

In [ ]:
draw_multivarient_plot(dataset=dataset_3c, rows=2, cols=3,plot_type="bar")

Yardımcı Fonksiyonlar 

get_models(): Kullanılacak makine öğrenmesi algoritmalarına geri dönen fonksiyon

get_X_and_y(): Veri setini 'veri(X)' ve 'çıktı(y)' olarak ayrıştırır.

accuracy_scores_for_various_ml_models(): Kullanılan makine öğrenmesi algoritmlarının başarı oranlarına geri döner

cross_validate(dataset, target_name): Var olan makine öğrenmesi algoritmaları için çapraz doğrulama sonuçlarını veren fonksiyon

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def get_models():
    models=[]
    models.append(("LR",LogisticRegression()))
    models.append(("LDA", LinearDiscriminantAnalysis()))
    models.append(("NB",GaussianNB()))
    models.append(("KNN",KNeighborsClassifier(n_neighbors = 3)))
    models.append(("DT",DecisionTreeClassifier()))
    models.append(("SVM rbf",SVC()))
    models.append(("SVM linear",SVC(kernel='linear')))
    
    return models

In [ ]:
def get_X_and_y(dataset, target_name):
    X=dataset.drop(target_name, axis=1)
    y=dataset[target_name]

    X=StandardScaler().fit_transform(X)
    
    labelEncoder=LabelEncoder()
    y=labelEncoder.fit_transform(y)
    
    return X, y

In [ ]:
def accuracy_scores_for_various_ml_models(X, y):
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
    models=get_models()
    for name, model in models:
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        score=model.score(X_test, y_test)
        
        print("{} accuracy:{:.2f}".format(name, score))
        print(confusion_matrix(y_test, pred))

In [ ]:
print("dataset_2c Veri Seti İçin Başarı Oranları")
X, y=get_X_and_y(dataset=dataset_2c, target_name='class')
accuracy_scores_for_various_ml_models(X, y)

In [ ]:
print("dataset_3c Veri Seti İçin Başarı Oranları")
X, y=get_X_and_y(dataset=dataset_3c, target_name='class')
accuracy_scores_for_various_ml_models(X, y)

In [ ]:
def cross_validation_scores_for_various_ml_models(X_cv, y_cv):
    print("Çapraz Doğrulama Başarı Oranları".upper())
    models=get_models()
    results=[]
    names= []

    for name, model in models:
        kfold=KFold(n_splits=5,random_state=22)
        cv_result=cross_val_score(model,X_cv, y_cv, cv=kfold,scoring="accuracy")
        names.append(name)
        results.append(cv_result)
        print("{} modelinin çapraz doğrulaması yapıldı, başarı oranı:{:0.2f}".format(name, cv_result.mean()))


def cross_validate(dataset, target_name):
    X, y=get_X_and_y(dataset, target_name)

    cross_validation_scores_for_various_ml_models(X, y)

In [ ]:
cross_validate(dataset=dataset_2c, target_name="class")

In [ ]:
def MY_SVM_GridSearch(dataset, target_name):
    X=dataset.drop(target_name, axis=1)
    y=dataset[target_name]
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
    best_score=0
    gammas=[0.001, 0.01, 0.1, 1, 10, 100]
    Cs=[0.001, 0.01, 0.1, 1, 10, 100]
    kernels=['rbf', 'linear']
    
    for gamma in gammas:
        for C in Cs:
            for kernel in kernels:
                svm=SVC(kernel=kernel,gamma=gamma, C=C)
                svm.fit(X_train, y_train)
                
                score=svm.score(X_test, y_test)

                if score>best_score:
                    y_pred=svm.predict(X_test)
                    best_score=score
                    best_params={'kernel':kernel, 'C':C, 'gamma':gamma}
        
    print("best score:",best_score)
    print("best params:",best_params)
    print("classification reports:\n",classification_report(y_test, y_pred))
    print("Karmaşıklık Matrisi:\n",confusion_matrix(y_test, y_pred))

In [ ]:
MY_SVM_GridSearch(dataset=dataset_2c, target_name='class')

In [ ]:
cross_validate(dataset=dataset_3c, target_name="class")

In [ ]:
scv_params=[{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

grid_search=GridSearchCV(SVC(), scv_params, cv=5)

In [ ]:
X, y=get_X_and_y(dataset=dataset_2c, target_name='class')
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
grid_search.fit(X_train, y_train)

print("İki Sınıf İçin sklearn GridSearchCV Sonuçları")
print()
print("best params:{}".format(grid_search.best_params_))

print()
test_means=grid_search.cv_results_['mean_test_score']
print("ortalama test sonucu:{:.2f}".format(np.mean(test_means)))

print()
y_pred=grid_search.predict(X_test)
print("en iyi parametre sonucu:{:.2f}".format(accuracy_score(y_test, y_pred)))
print("classification report:\n{}".format(classification_report(y_test, y_pred)))
print("Karmaşıklık Matrisi:\n",confusion_matrix(y_test, y_pred))

In [ ]:
X, y=get_X_and_y(dataset=dataset_3c, target_name='class')
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
grid_search.fit(X_train, y_train)

print("Üç Sınıf İçin sklearn GridSearchCV Sonuçları")
print()
print("best params:{}".format(grid_search.best_params_))

print()
test_means=grid_search.cv_results_['mean_test_score']
print("ortalama test sonucu:{:.2f}".format(np.mean(test_means)))

print()
y_pred=grid_search.predict(X_test)
print("en iyi parametre sonucu:{:.2f}".format(accuracy_score(y_test, y_pred)))
print("classification report:\n{}".format(classification_report(y_test, y_pred)))
print("Karmaşıklık Matrisi:\n",confusion_matrix(y_test, y_pred))